In [1]:
import os
import sys
import sqlite3
import numpy as np
import pandas as pd
import geopandas as gp
import plotly.express as px
import matplotlib.pyplot as plt
import scipy.stats as scs

import pyet
import pyeto
import spei

In [2]:
sys.path.append('/home/pooya/w/DroughtMonitoringIran/')

DATABASE_PATH = "./database/database.db"

### Monthly Data

In [3]:
conn = sqlite3.connect(DATABASE_PATH)

geoinfo = pd.read_sql(sql='SELECT * FROM geoinfo', con=conn)

ground_data = pd.read_sql(sql='SELECT * FROM monthly', con=conn)
ground_data['Date'] = pd.to_datetime(ground_data['Date'])
ground_data["Date"] = ground_data["Date"].dt.to_period("M").astype(str)

pet_data = pd.read_sql(sql='SELECT * FROM pet_monthly', con=conn)
pet_data['Date'] = pd.to_datetime(pet_data['Date'])
pet_data["Date"] = pet_data["Date"].dt.to_period("M").astype(str)

si_data = pd.read_sql(sql='SELECT * FROM si_monthly', con=conn)
si_data['Date'] = pd.to_datetime(si_data['Date'])
si_data["Date"] = si_data["Date"].dt.to_period("M").astype(str)

precip_data = pd.read_sql(sql='SELECT * FROM precip_monthly', con=conn)

vi_data = pd.read_sql(sql='SELECT * FROM vi_monthly', con=conn)

lst_data = pd.read_sql(sql='SELECT * FROM lst_monthly', con=conn)

conn.close()

In [4]:
data = geoinfo.copy()

data = data\
    .merge(ground_data, on='Station_ID', how='outer')\
    .merge(pet_data, on=['Station_ID', 'Date'], how='outer')\
    .merge(si_data, on=['Station_ID', 'Date'], how='outer')\
    .merge(precip_data, on=['Station_ID', 'Date'], how='outer')\
    .merge(vi_data, on=['Station_ID', 'Date'], how='outer')\
    .merge(lst_data, on=['Station_ID', 'Date'], how='outer')\
    .query('Date >= "2006-01" and Date <= "2024-06"')\
    .reset_index(drop=True)

data

,Station_ID,Station_Name,Province,Station_Latitude,Station_Longitude,Station_Elevation,Date,Temp_Max,Temp_Min,Temp_Mean,...,PERSIANNCDR_Precip,CHIRPS_Precip,NDVI_MOD13A3,NDVI_MYD13A3,EVI_MOD13A3,EVI_MYD13A3,LSTDay_MOD21C3,LSTDay_MYD21C3,LSTNight_MOD21C3,LSTNight_MYD21C3
0,40709,Astara,Gilan,38.365,48.855,-21.1,2006-01,6.8,0.9,3.8,...,83.969995,NaN,0.4313,0.4250,0.1815,0.1773,9.09,6.11,7.53,1.31
1,40709,Astara,Gilan,38.365,48.855,-21.1,2006-02,9.2,3.2,6.2,...,49.962416,NaN,0.3934,0.4522,0.1602,0.1800,11.77,6.77,5.39,3.19
2,40709,Astara,Gilan,38.365,48.855,-21.1,2006-03,13.2,6.8,10.0,...,38.709023,NaN,0.3795,0.4180,0.1754,0.1841,17.67,10.71,8.33,7.03
3,40709,Astara,Gilan,38.365,48.855,-21.1,2006-04,16.5,10.9,13.7,...,86.245682,NaN,0.4790,0.4157,0.2203,0.1671,25.09,14.87,9.91,8.23
4,40709,Astara,Gilan,38.365,48.855,-21.1,2006-05,21.0,14.7,17.9,...,41.803581,NaN,0.6411,0.4320,0.3021,0.2168,28.01,19.57,17.51,16.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5545,99361,Alasht,Mazandaran,36.071,52.843,1805.0,2024-02,7.5,-1.3,3.1,...,49.011868,285.324780,0.3920,0.2661,0.1730,0.1344,0.11,14.45,2.21,-1.47
5546,99361,Alasht,Mazandaran,36.071,52.843,1805.0,2024-03,12.1,1.8,6.9,...,24.582743,71.169762,-999.0000,0.3203,-999.0000,0.1551,5.63,22.47,3.73,-0.07
5547,99361,Alasht,Mazandaran,36.071,52.843,1805.0,2024-04,15.7,6.2,10.9,...,NaN,43.918756,0.3901,0.7398,0.2489,0.5179,21.71,26.89,7.97,2.63
5548,99361,Alasht,Mazandaran,36.071,52.843,1805.0,2024-05,18.0,9.3,13.6,...,NaN,52.235768,0.5332,0.6308,0.3797,0.4374,22.71,30.37,13.01,6.59


In [5]:
conn = sqlite3.connect(DATABASE_PATH)

data.to_sql('data', conn, if_exists='replace', index=False)
conn.commit()

conn.close()